In [1]:
pip install dash pandas plotly


   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.8 MB 435.7 kB/s eta 0:00:18
   ---------------------------------------- 0.0/7.8 MB 326.8 kB/s eta 0:00:24
   ---------------------------------------- 0.1/7.8 MB 525.1 kB/s eta 0:00:15
    --------------------------------------- 0.1/7.8 MB 554.9 kB/s eta 0:00:14
    --------------------------------------- 0.2/7.8 MB 737.3 kB/s eta 0:00:11
   - -------------------------------------- 0.2/7.8 MB 801.7 kB/s eta 0:00:10
   - -------------------------------------- 0.3/7.8 MB 803.7 kB/s eta 0:00:10
   - -------------------------------------- 0.4/7.8 MB 916.6 kB/s eta 0:00:09
   -- ------------------------------------- 0.4/7.8 MB 890.4 kB/s eta 0:00:09
   -- ------------------------------------- 0.5/7.8 MB 994.4 kB/s eta 0:00:08
   --- ------------------------------------ 0.6/7.8 MB 1.2 MB/s eta 0:00:07
   ---

In [3]:

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Set the title of the dashboard
#app.title = "Automobile Statistics Dashboard"

#---------------------------------------------------------------------------------
# Create the dropdown menu options
dropdown_options = [
    {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]
# List of years 
year_list = [i for i in range(1980, 2024, 1)]
#---------------------------------------------------------------------------------------
# Create the layout of the app
app.layout = html.Div([
    #TASK 2.1 Add title to the dashboard
    html.H1("Automobile Sales Statistics Dashboard", style = {"textAlign":"center","color":"#503D36", "font-size":24}),#May include style for title
    html.Div([#TASK 2.2: Add two dropdown menus
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=[{'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                           {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
                           ],
            value='Select Statistics',
            placeholder='Select a report type'
        )
    ]),html.Br(),

    html.Div([
        html.Label("Select Year:"),
        dcc.Dropdown(
            id='select-year',
            options=[{'label': i, 'value': i} for i in year_list],
            value='Select Statistics'
        )]),
    html.Div([#TASK 2.3: Add a division for output display
    html.Div(id='output-container', className='chart-grid', 
    style={"display":"flex"}),])
])
#TASK 2.4: Creating Callbacks
# Define the callback function to update the input container based on the selected statistics
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics', component_property='value'),
     
)
def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True

#Callback for plotting
# Define the callback function to update the input container based on the selected statistics
# Callback function
# Callback function
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'), 
    Input(component_id='select-year', component_property='value')])
def update_output_container(selected_statistics, input_year):
    global data  # Ensure data is accessible within the callback

    if selected_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]

        # Plot 1: Automobile sales fluctuate over the Recession Period (year-wise)
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(
                yearly_rec,
                x='Year',
                y='Automobile_Sales',
                title="Average Automobile Sales fluctuation over Recession Period"
            )
        )

        # Plot 2: Calculate the average number of vehicles sold by vehicle type
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(figure=px.pie(
            average_sales,
            values="Automobile_Sales",
            names="Vehicle_Type",
            title="Automobile Sales with respect to Vehicle Type"
        ))

        # Plot 3: Pie chart for total expenditure share by vehicle type during recessions
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].mean().reset_index()
        R_chart3 = dcc.Graph(figure=px.pie(
            exp_rec,  # Pass the DataFrame here
            values="Advertising_Expenditure",
            names="Vehicle_Type",
            title="Advertising Expenditure with respect to Vehicle Type"
        ))

        # Plot 4: Bar chart for the effect of the unemployment rate on vehicle type and sales
        unempl_sales = recession_data.groupby(['Vehicle_Type', "unemployment_rate"])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(figure=px.bar(
            unempl_sales,
            x="Vehicle_Type",
            y="Automobile_Sales",
            title="Automobile Sales with respect to Vehicle Type"
        ))

        return [
            html.Div(className='chart-grid', children=[html.Div(R_chart1), html.Div(R_chart2)],
                     style={"display": "flex-1"}),
            html.Div(className='chart-grid', children=[html.Div(R_chart3), html.Div(R_chart4)],
                     style={"display": "flex-2"})
        ]


    # Yearly Statistic Report Plots
    elif input_year != 'Select Statistics' and selected_statistics == 'Yearly Statistics':
        yearly_data = data[data['Year'] == int(input_year)]

        # Plot 1: Yearly Automobile sales using line chart for the whole period.
        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(
            yas,
            x='Year',
            y='Automobile_Sales',
            title="Yearly Average Automobile Sales fluctuation over Whole Period"
        ))

        # Plot 2: Total Monthly Automobile sales using line chart.
        m_sale = data.groupby('Month')['Automobile_Sales'].mean().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(
            m_sale,
            x='Month',
            y='Automobile_Sales',
            title="Monthly Average Automobile Sales"
        ))

        # Plot 3: Bar chart for unemployment number of vehicles sold during the given year
        avr_vdata = yearly_data.groupby(["unemployment_rate", "Year"])["Automobile_Sales"].sum().reset_index()
        Y_chart3 = dcc.Graph(figure=px.bar(
            avr_vdata,
            x="unemployment_rate",
            y="Automobile_Sales",
            title=f'Average Vehicles Sold by Vehicle Type in the year {input_year}'
        ))

        # Plot 4: Total Advertisement Expenditure for each vehicle using pie chart
        exp_data = yearly_data.groupby("Vehicle_Type")["Advertising_Expenditure"].sum().reset_index()
        Y_chart4 = dcc.Graph(figure=px.pie(
            exp_data,
            values="Advertising_Expenditure",
            names="Vehicle_Type",
            title="Total Advertising Expenditure with respect to Vehicle Type"
        ))
        return [
            html.Div(className='chart-grid', children=[html.Div(Y_chart1), html.Div(Y_chart2)],
                     style={"display": "flex-1"}),
            html.Div(className='chart-grid', children=[html.Div(Y_chart3), html.Div(Y_chart4)],
                     style={"display": "flex-2"})
        ]

    else:
        return None

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug = True)

